# PDF 문서를 image로 변환 후 요약하는 샘플 코드

In [1]:
!pip install PyMuPDF Pillow boto3

## 1. 매서드 작성

In [2]:
import fitz  # PyMuPDF
import boto3
import base64
from PIL import Image
import io
import json

In [3]:
# 1. PDF를 이미지로 변환
def pdf_to_images(pdf_path):
    doc = fitz.open(pdf_path)
    images = []
    for page in doc:
        pix = page.get_pixmap()
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
        images.append(img)
    return images

In [4]:
# 2. 이미지를 Base64로 인코딩
def image_to_base64(image):
    buffered = io.BytesIO()
    image.save(buffered, format="PNG")
    return base64.b64encode(buffered.getvalue()).decode()

In [5]:
# 3. Amazon Bedrock 클라이언트 설정
bedrock = boto3.client(
    service_name='bedrock-runtime',
    region_name='us-east-1'  # 예: 'us-east-1'
)

In [20]:
# 4. Claude 모델에 이미지 전송 및 응답 받기
def get_claude_response(image_base64):
    
    prompt = """
    다음 <instruction></instruction>에 따라 이미지의 context를 최대한 자세히 알려주세요.
    <instruction>
    1. 중요 정보나 객관적 데이터를 누락하지 말고 반드시 포함하세요.
    2. Content 또는 Agenda 같은 목차에 대한 내용은 보이는 그대로 자세하게 적어 주세요. 세부 목차가 있다면 생략하지 말고 전부 적어주세요.
    3. 테이블이나 차트와 같은 부분에 대해서도 자세한 내용을 포함하세요.
    4. 타이틀이 있는 내용은 타이틀 이름도 반드시 포함하세요.
    </instruction>
    """
    
    body = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 4096,
        "temperature": 0,
        "top_p": 0,
        "top_k": 0,
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/png",
                            "data": image_base64
                        }
                    },
                    {
                        "type": "text",
                        "text": prompt
                    }
                ]
            }
        ]
    }

    #테스트 환경에 Claude 3.5 사용이 불가합니다. 고객 분들은 아래 modelId를 Claude 3.5 Sonnet 으로 변경해주세요.
    response = bedrock.invoke_model(
        body=json.dumps(body),
        #modelId = "anthropic.claude-3-5-sonnet-20240620-v1:0"  # Claude 3.5 Sonnet
        modelId="anthropic.claude-3-sonnet-20240229-v1:0",      # Claude 3 Sonnet
        contentType="application/json",
        accept="application/json"
    )

    response_body = json.loads(response['body'].read())
    return response_body['content'][0]['text']

## 2. Claude 3.x 모델 호출하여 이미지 pdf 처리

#### Claude 3.5 Sonnet 의 이미지 분석 성능이 더 뛰어납니다. 아래 결과는 Claude 3 Sonnet 결과 입니다,

In [21]:
pdf_path = "./rag_data/Steel Business Briefing_04 Jul 2024.pdf"
images = pdf_to_images(pdf_path)

img_context = []

for i, image in enumerate(images):
    image_base64 = image_to_base64(image)
    response = get_claude_response(image_base64)
    print(f"Page {i+1} Summary:")
    print(response)
    img_context.append(response)
    print("\n---\n")

Page 1 Summary:
이 이미지는 SBB Global Commodity Insights의 일일 브리핑 문서로 보입니다. 상단에는 Platts와 Platts Connect에 대한 정보가 있습니다.

Price snapshot 섹션에는 다양한 상품의 심볼, 단위, 가격, 변동, 변동률, 평가 일자가 테이블 형식으로 제시되어 있습니다. 상품으로는 Dubai 부탄, Euro 유로, NWE 등이 있습니다.

Weekly prices 섹션에는 HSFO FO 380 cSt의 가격이 제시되어 있습니다.

Contents 섹션에는 Asia, Fuel Products, Bunker, Crude, EMEA, Fuel Products, Petrochemicals, LPG, NGL, Biofuels, Dry Freight, CIS 등의 주제별 세부 목차가 나열되어 있습니다.

---

Page 2 Summary:
이 이미지는 SBB(Steel, Billets, Blooms) Daily Briefing 보고서의 일부분입니다. 상단에는 "Asia", "Flat Products", "Asian HRC at standstill; buying indications fading behind"라는 제목이 있습니다.

아시아 지역 내용에는 다음과 같은 세부사항이 포함되어 있습니다:

- 중국 국내 가격은 톤당 3,850위안으로 전일 대비 2% 하락
- 중국 수출 가격은 톤당 $520/CFR로 전일과 동일
- 동남아시아 수출 가격도 톤당 $520/CFR로 전일과 동일
- 일본/한국 코일 가격은 톤당 $510/CFR 수준으로 추정
- 아시아 HRC 시장은 거래 정체 상태, 구매 관심 감소 조짐

"Flat Products" 섹션에서는 중국, 일본, 한국의 평판 제품 가격 동향을 설명하고 있습니다.

하단에는 "Indian rebar prices dip as mills lower offers to ignite market; HRC mills compete" 제목 아래 인도 철강 시장 동향이 요약

### img_context 내용을 다음 노트북에서 사용하도록 저장

In [22]:
%store img_context

Stored 'img_context' (list)


In [23]:
img_context

['이 이미지는 SBB Global Commodity Insights의 일일 브리핑 문서로 보입니다. 상단에는 Platts와 Platts Connect에 대한 정보가 있습니다.\n\nPrice snapshot 섹션에는 다양한 상품의 심볼, 단위, 가격, 변동, 변동률, 평가 일자가 테이블 형식으로 제시되어 있습니다. 상품으로는 Dubai 부탄, Euro 유로, NWE 등이 있습니다.\n\nWeekly prices 섹션에는 HSFO FO 380 cSt의 가격이 제시되어 있습니다.\n\nContents 섹션에는 Asia, Fuel Products, Bunker, Crude, EMEA, Fuel Products, Petrochemicals, LPG, NGL, Biofuels, Dry Freight, CIS 등의 주제별 세부 목차가 나열되어 있습니다.',
 '이 이미지는 SBB(Steel, Billets, Blooms) Daily Briefing 보고서의 일부분입니다. 상단에는 "Asia", "Flat Products", "Asian HRC at standstill; buying indications fading behind"라는 제목이 있습니다.\n\n아시아 지역 내용에는 다음과 같은 세부사항이 포함되어 있습니다:\n\n- 중국 국내 가격은 톤당 3,850위안으로 전일 대비 2% 하락\n- 중국 수출 가격은 톤당 $520/CFR로 전일과 동일\n- 동남아시아 수출 가격도 톤당 $520/CFR로 전일과 동일\n- 일본/한국 코일 가격은 톤당 $510/CFR 수준으로 추정\n- 아시아 HRC 시장은 거래 정체 상태, 구매 관심 감소 조짐\n\n"Flat Products" 섹션에서는 중국, 일본, 한국의 평판 제품 가격 동향을 설명하고 있습니다.\n\n하단에는 "Indian rebar prices dip as mills lower offers to ignite market; HRC mills compete" 제목 아래 인도 철강 시장 동향이 요약되어 있습니다.',
 '이